In [1]:
import pandas as pd
import numpy as np
import csv
import pickle

In [2]:
sample  = pickle.load( open( "data/twitter_data_clean.p", "rb" ) )
sample.head()

,city,country,day,followersCount,friendsCount,hour,id,inReplyTo,latitude,longitude,month,placeLatitude,placeLongitude,state,statusesCount,text,userId,userLocation,week_number,year
551,Binningen,ch,9.0,14249,9260.0,18.0,10231423626,NaN,47.5455,7.58531,3.0,47.5367,7.57849,Basel-Landschaft,19585.0,"The new apartment is nice, but there is no Wif...",6.25728e+06,"Potsdam, Germany",10.0,2010.0
609,Zürich,ch,10.0,177,136.0,22.0,10292646240,NaN,47.3876,8.52725,3.0,47.3791,8.50021,Zürich,5167.0,Is that wet yet solid stuff on my screen suppo...,1.5602e+07,"Zürich, Switzerland",10.0,2010.0
611,Genève,ch,11.0,471,82.0,5.0,10309829732,NaN,46.2006,6.13183,3.0,46.1996,6.13011,Genève,3363.0,I'm at DCTI - David Dufour in Geneva http://go...,625553,"Geneva, Switzerland",10.0,2010.0
612,Köniz,ch,11.0,586,508.0,6.0,10310391132,NaN,46.8957,7.44235,3.0,46.9214,7.38855,Bern - Berne,9016.0,God morgon! :-),1.7341e+07,"Bern, Switzerland",10.0,2010.0
618,Genève,ch,11.0,2230,387.0,7.0,10311568050,NaN,46.2043,6.19900,3.0,46.1938,6.15415,Genève,10605.0,"At this very minute, the sun is pink.",634553,"Geneva, Switzerland",10.0,2010.0


In [3]:
sample.isnull().sum()

city                 88225
country                 24
day                      1
followersCount         148
friendsCount           161
hour                     0
id                       0
inReplyTo         11255929
latitude           5760885
longitude          5760885
month                    1
placeLatitude            0
placeLongitude           0
state                  274
statusesCount          167
text                149864
userId                   0
userLocation       4041308
week_number              1
year                     1
dtype: int64

In [4]:
sample.astype

<bound method NDFrame.astype of                       city country   day followersCount  friendsCount  hour  \
551              Binningen      ch   9.0          14249        9260.0  18.0   
609                 Zürich      ch  10.0            177         136.0  22.0   
611                 Genève      ch  11.0            471          82.0   5.0   
612                  Köniz      ch  11.0            586         508.0   6.0   
618                 Genève      ch  11.0           2230         387.0   7.0   
622                 Genève      ch  11.0            167         277.0  11.0   
623                  Basel      ch  11.0          14249        9260.0  11.0   
630                 Zürich      ch  11.0            230         276.0  13.0   
632       Birmensdorf (ZH)      ch  11.0            533         292.0  15.0   
633                 Zürich      ch  11.0            394         266.0  16.0   
634                 Zürich      ch  11.0             75         161.0  17.0   
635                 

A few times, countries and states is not found, so we can drop it.

In [6]:
user_all=sample.dropna(subset=['country','state','year','month','week_number','day','hour'])
user_all.isnull().sum()

city                 88197
country                  0
day                      0
followersCount         148
friendsCount           161
hour                     0
id                       0
inReplyTo         11255701
latitude           5760885
longitude          5760885
month                    0
placeLatitude            0
placeLongitude           0
state                    0
statusesCount          167
text                149862
userId                   0
userLocation       4041218
week_number              0
year                     0
dtype: int64

The essential fields like 'placeLatitude' and 'placeLongtitude' are clean, so we can filter Switzerland's data for analysis now.

In [7]:
user=user_all.loc[user_all['country']=='ch']
user.shape

(10129731, 20)

# Cleaning unusual trajectories

Revocer time for calculating valid user in reasonable moving speed.

In [8]:
import time
import datetime

t_count=user[['year','month','day','hour']]
temp=pd.to_datetime(t_count)
temp.head()
user['time']=temp
user.head()

D:\anaconda\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,city,country,day,followersCount,friendsCount,hour,id,inReplyTo,latitude,longitude,...,placeLatitude,placeLongitude,state,statusesCount,text,userId,userLocation,week_number,year,time
551,Binningen,ch,9.0,14249,9260.0,18.0,10231423626,NaN,47.5455,7.58531,...,47.5367,7.57849,Basel-Landschaft,19585.0,"The new apartment is nice, but there is no Wif...",6.25728e+06,"Potsdam, Germany",10.0,2010.0,2010-03-09 18:00:00
609,Zürich,ch,10.0,177,136.0,22.0,10292646240,NaN,47.3876,8.52725,...,47.3791,8.50021,Zürich,5167.0,Is that wet yet solid stuff on my screen suppo...,1.5602e+07,"Zürich, Switzerland",10.0,2010.0,2010-03-10 22:00:00
611,Genève,ch,11.0,471,82.0,5.0,10309829732,NaN,46.2006,6.13183,...,46.1996,6.13011,Genève,3363.0,I'm at DCTI - David Dufour in Geneva http://go...,625553,"Geneva, Switzerland",10.0,2010.0,2010-03-11 05:00:00
612,Köniz,ch,11.0,586,508.0,6.0,10310391132,NaN,46.8957,7.44235,...,46.9214,7.38855,Bern - Berne,9016.0,God morgon! :-),1.7341e+07,"Bern, Switzerland",10.0,2010.0,2010-03-11 06:00:00
618,Genève,ch,11.0,2230,387.0,7.0,10311568050,NaN,46.2043,6.19900,...,46.1938,6.15415,Genève,10605.0,"At this very minute, the sun is pink.",634553,"Geneva, Switzerland",10.0,2010.0,2010-03-11 07:00:00


Check and drop NaT (for our calculation accurately in hour, something bad may happen).

In [9]:
# velticity calculation, miles per hour
from geopy.distance import vincenty 

def speed(loc1, loc2, t1, t2):
    d = vincenty(loc1,loc2).miles
    dt = t2-t1
    dh = dt.seconds/3600.0 #to_hours
    if dh :
        return d/dh 
    else: #within one hour, assume it travels within half an hour
        return d/0.5

In [10]:
#ref: Exploring Multi-Scale Spatiotemporal Twitter User Mobility Patterns with a Visual-Analytics Approach
#Space-Time Twitter User Trajectories
#To remove non-human users based on unusual relocating speed: e.g. speed (loc_i − loc_i−1) > 240 m/s

#Travelling in Switzerland mainly takes train(despite some ones take flight)
#Data are in time-order, so the location is continuously changed
#Since we clean the data before, time is accurate to hour, so we set a loose thedshold like 200 mils/h 


#remove non-human user
def human_user(c_tuple, speed_thed=200):
    lon,lan,t = tuple(zip(*c_tuple))
    point = list(zip(lan,lon))
    normal=[True]*(len(c_tuple))
    
    if len(c_tuple) == 1:
        normal = [False]*(len(c_tuple)) # remove only one tweet
    if len(c_tuple) == 2 :
        v = speed(point[0],point[1],t[0],t[1])
        if v > speed_thed:
            normal = [False]*(len(c_tuple))
    else:
        loc1 = point[0:-2]
        loc2 = point[1:-1]
        loc3 = point[2::]
        #Using geopy's vincenty to find distances in pandas data frame
        for i in range(len(loc1)):
            v1_2=speed(loc2[i],loc1[i],t[i],t[i+1])
            v2_3=speed(loc2[i],loc3[i],t[i+1],t[i+2])
            v1_3=speed(loc1[i],loc3[i],t[i],t[i+2])
        
            # Check if loc1 is a normal location with reasonable speed
            # If there is something wrong with loc1->loc2, we check loc2->loc3 & loc1->loc3 to ensure if loc2/3 is wrong
        
            if v1_2 > speed_thed: # loc1->loc2 innormal
                if v2_3 > speed_thed: # loc2->loc1/3 innormal -> loc2 innormal
                    normal[i+1]=False 
                    if v1_3 > speed_thed: # loc1->loc2/3 innormal -> loc1 innormal
                        normal[i]=False 
                        if i == len(loc1)-1: # last index with all wrong
                            normal[i+2]=False
                else: # loc1->loc2 innormal, loc2->loc1/3 normal -> loc2 normal, loc1 innormal
                    normal[i]=False
            else: # loc1->loc2 normal
                if v2_3 > speed_thed:
                    if v1_3 > speed_thed: # loc3->loc1/2 innormal -> loc3 innormal
                        normal[i+2]=False
                    else:
                        if (i == len(loc1)-1): # last index, just loc2->loc3 innormal -> loc2 innormal
                            normal[i+1]=False
        
    return normal

In [11]:
#twitter_user=twitter_data.groupby(twitter_data['userId'])
#.order(ascending=True)
user_trace=['userId','year','month','week_number','day']
user['temp'] = tuple(zip(user['placeLatitude'],user['placeLongitude'],user['time']))
normal = user.groupby(by=user_trace)['temp'].transform(lambda x: human_user(x))

D:\anaconda\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
normal.shape

(10129731,)

In [14]:
user.shape

(10129731, 22)

In [15]:
normal_user = user[normal].reset_index(drop=True)
normal_user.shape

(9044734, 22)

# Store trace

Next,we hope to define daily trace of every user.

In [16]:
# Since tweets are in time order, we can just store daily trace by apprending locations(canton)

def daily_trace(r_location):
    start=r_location.values[0:-1]
    end=r_location.values[1::]
    index=(start!=end)
    trace=np.nan
    
    if index.any(): # Remove inner-canton trace 
        trace='->'.join(start[index])
        trace='->'.join([trace,end[index][-1]])
        
    return trace    
    

In [17]:
user_dtrace = normal_user.groupby(by=user_trace)['state'].apply(lambda x: daily_trace(x))


In [18]:
user_dtrace=user_dtrace.dropna()
user_dtrace.head()

userId  year    month  week_number  day 
246.0   2012.0  12.0   51.0         23.0                     Luzern->Uri
1102.0  2013.0  12.0   52.0         26.0             Zürich->Uri->Zürich
1497.0  2015.0  9.0    38.0         15.0    Bern - Berne->Aargau->Zürich
1775.0  2015.0  6.0    25.0         21.0                  Ticino->Zürich
2397.0  2011.0  10.0   42.0         18.0                    Genève->Vaud
Name: state, dtype: object

In [19]:
with open ( "state_mov_all.pkl", "wb" ) as output:
    pickle.dump(temp, output, pickle.HIGHEST_PROTOCOL)

In [20]:
user_dtrace.shape

(120960,)

In [22]:
user_dtrace.to_csv('daily_trace1.csv')